## 1. 사용환경 준비
`OpenAI` 모델

In [1]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

OpenAI API key 입력:  ········


## 2. 모델 로드하기
`OpenAI` 모델을 로드하여 model에 저장

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

## 3. 문서 로드하기
`langchain`의 `PyPDFLoader` 를 이용하여 문서를 불러옴

In [5]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("C:/Users/jh838/인공지능산업최신동향_2024년11월호.pdf")

# 페이지별 문서 로드
docs = loader.load()

## 4. 문서 청크로 나누기
### 4-1. CharacterTextSplitter
텍스트를 지정한 길이의 청크로 단순히 분할

In [18]:
from langchain.text_splitter import CharacterTextSplitter

# CharacterTextSplitter로 청킹
character_splitter = CharacterTextSplitter(
    separator="\n\n",         # 두 줄 개행을 기준으로 분리
    chunk_size=100,           # 각 청크의 최대 길이
    chunk_overlap=10,         # 청크 간의 겹치는 길이
    length_function=len,      # 길이를 측정하는 함수
    is_separator_regex=False  # separator를 정규식으로 사용하지 않음
)
character_chunks = character_splitter.split_documents(docs)

# 청킹된 결과의 상위 10개 출력
print("=== CharacterTextSplitter 모델 응답 상위 10개 ===")
for i, chunk in enumerate(character_chunks[:10]):
    messages = [HumanMessage(content=chunk.page_content)]
    response = model(messages)
    print(f"Character Chunk {i+1} Response:\n{response.content}\n")

=== CharacterTextSplitter 모델 응답 상위 10개 ===
Character Chunk 1 Response:
2024년 11월호에 대한 구체적인 내용이나 주제가 필요하신가요? 예를 들어, 특정 잡지나 기사, 혹은 이벤트와 관련된 정보가 필요하신지 말씀해 주시면 더 도움을 드릴 수 있습니다!

Character Chunk 2 Response:
2024년 11월호 인공지능 산업 동향 브리프는 다양한 주제를 다루고 있습니다. 아래는 각 섹션의 주요 내용을 요약한 것입니다.

### Ⅰ. 인공지능 산업 동향 브리프

#### 1. 정책/법제
- **미국 민권위원회**: 연방정부의 얼굴인식 기술 사용이 민권에 미치는 영향 분석.
- **백악관 예산관리국**: 정부의 책임 있는 AI 조달을 위한 새로운 지침 발표.
- **유로폴**: 법 집행에서 AI의 장점과 과제를 분석한 보고서 발간.
- **OECD**: 공공 부문 AI 도입을 위한 G7 툴킷 발표.
- **세계경제포럼**: 생성AI 시대의 거버넌스 프레임워크 제시.

#### 2. 기업/산업
- **CB인사이츠**: 2024년 3분기 벤처 투자의 31%가 AI 스타트업으로 집중.
- **메타**: 동영상 생성AI 도구 ‘메타 무비 젠’ 출시 및 첫 멀티모달 AI 모델 ‘라마 3.2’ 발표.
- **앨런AI연구소**: GPT-4o를 초월하는 성능의 오픈소스 LLM ‘몰모’ 공개.
- **미스트랄AI**: 온디바이스 AI 모델 ‘레 미니스트로’ 발표.
- **카카오**: 통합 AI 브랜드 및 신규 AI 서비스 ‘카나나’ 공개.

#### 3. 기술/연구
- **노벨상 수상**: AI 관련 연구자들이 2024년 노벨 물리학상 및 화학상 수상.
- **미국 국무부**: AI 연구에서의 국제협력을 위한 ‘글로벌 AI 연구 의제’ 발표.
- **일본 AI안전연구소**: AI 안전성 평가 관점 가이드 발간.
- **구글 딥마인드**: 반도체 칩 레이아웃 설계 AI 모델 ‘알파칩’ 발표.
- **AI21 CEO

### CharacterTextSplitter 파라미터 설명
`separator`: 문자열을 분할하는 기준이 되는 구분자. `\n\n`로 설정하면 두 줄 개행마다 텍스트를 나누게 됨.

`chunk_size`: 각 청크의 최대 길이로, 지정된 길이 이상의 문자는 하나의 청크에 포함도지 않게 함. `100`으로 설정되어 각 청크가 최대 100자 길이가 됨

`chunk_overlap`: 청크 간에 겹치는 길이를 설정. `10`으로 설정하면 각 청크가 서로 겹치는 10자를 공유

`length_function`: 청크의 길이를 측정하는 함수로,`len` 함수를 사용하여 문자열의 길이를 계산

`is_separator_regex`: 구분자를 정규식으로 사용할지 여부를 설정. `True`로 설정하면 `separator` 값을 정규식으로 인식하여 문자열을 나누고, `False`로 설정할 경우`separator`는 단순한 문자열로 사용
서 사용할 수 있습니다.


### 4-2. RecursiveCharacterTextSplitter
텍스트를 재귀적으로 청킹하여 텍스트 구조를 보존하면서 분할

파라미터 해설은 위와 동일

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter로 청킹
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,          # 각 청크의 최대 길이
    chunk_overlap=10,        # 청크 간의 겹치는 길이
    length_function=len,     # 길이를 측정하는 함수
    is_separator_regex=False # separator를 정규식으로 사용하지 않음
)
recursive_chunks = recursive_text_splitter.split_documents(docs)

# 청킹된 결과의 상위 10개 출력
print("=== RecursiveCharacterTextSplitter 모델 응답 상위 10개 ===")
for i, chunk in enumerate(recursive_chunks[:10]):
    messages = [HumanMessage(content=chunk.page_content)]
    response = model(messages)
    print(f"Recursive Chunk {i+1} Response:\n{response.content}\n")

=== RecursiveCharacterTextSplitter 모델 응답 상위 10개 ===
Recursive Chunk 1 Response:
죄송하지만 2024년 11월호에 대한 구체적인 정보나 내용을 제공할 수는 없습니다. 하지만 특정 주제나 질문이 있다면, 그에 대해 도와드릴 수 있습니다. 어떤 내용을 원하시는지 말씀해 주시면 최선을 다해 도와드리겠습니다!

Recursive Chunk 2 Response:
죄송하지만, 2024년 11월호에 대한 정보는 제공할 수 없습니다. 그 시점 이후의 자료는 제가 알지 못합니다. 다른 질문이나 요청이 있으시면 도와드리겠습니다!

Recursive Chunk 3 Response:
Ⅰ. 인공지능 산업 동향 브리프

1. 정책/법제  
   ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석

최근 미국 민권위원회(US Commission on Civil Rights)는 연방정부의 얼굴인식 기술 사용이 개인의 민권에 미치는 영향을 분석한 보고서를 발표했습니다. 이 보고서는 얼굴인식 기술이 인종적 편견과 차별을 초래할 수 있는 가능성을 지적하며, 이러한 기술의 사용이 인권을 침해할 수 있는 여러 측면을 다루고 있습니다.

주요 내용은 다음과 같습니다:

- **인종적 편향**: 얼굴인식 기술이 특정 인종이나 성별에 대해 정확도가 떨어지는 경우가 많아, 경찰 및 공공기관에서의 사용이 인종적 차별로 이어질 수 있다는 우려가 제기되었습니다.

- **프라이버시 침해**: 얼굴인식 기술이 개인의 동의 없이 사용될 경우, 개인의 프라이버시가 침해될 수 있으며, 이는 법적 및 윤리적 문제를 초래할 수 있습니다.

- **법적 규제 필요성**: 보고서는 얼굴인식 기술의 사용에 대한 명확한 법적 규제가 필요하다고 강조하며, 각 주 및 연방 차원에서의 규제 방안을 모색할 것을 권장했습니다.

- **투명성 및 책임**: 얼굴인식 기술을 사용하는 기관들은 기술의 사용 목적과 방법에 대한 투명성을 제공해야 하며, 기술이

## 5. 벡터 임베딩 생성
`OpenAIEmbeddings`을 이용해 텍스트를 벡터로 변환할 벡터 임베딩을 생성

In [45]:
from langchain.embeddings import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

C:\Users\jh838\AppData\Local\Temp\ipykernel_1920\1842267153.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


## 6. 벡터 스토어 생성
앞서 만든 벡터 임베딩과 청크된 문서를 활용하여 `FAISS` 벡터 스토어를 생성

In [50]:
import faiss
from langchain.vectorstores import FAISS

# 청크된 문서와 생성된 벡터 임베딩을 활용하여 FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(documents=recursive_chunks, embedding=embeddings)

# 벡터 스토어가 잘 생성되었는지 확인
print("FAISS 벡터 스토어가 성공적으로 생성되었습니다.")

FAISS 벡터 스토어가 성공적으로 생성되었습니다.


## 7. FAISS를 Retriever로 변환
RAG 체인에서 사용할 수 있도록 `FAISS`를 `retriever`로 변환

In [55]:
# FAISS 벡터 스토어를 retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# 변환된 retriever가 잘 작동하는지 확인
print("FAISS retriever가 성공적으로 생성되었습니다.")

FAISS retriever가 성공적으로 생성되었습니다.


## 8. 프롬프트 템플릿 정의

In [63]:
from langchain.prompts import ChatPromptTemplate

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\n\nQuestion: {question}")
])

# 템플릿 확인
print("Contextual prompt template has been successfully defined.")

Contextual prompt template has been successfully defined.


## 9. RAG 체인 구성
`LangChain`의 모델과 프롬프트를 연결하여 `RAG`체인을 구성

In [67]:
from langchain.chains import LLMChain

# 간단한 패스스루 클래스 정의
class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

# ContextToPrompt 클래스 정의
class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# RetrieverWrapper 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs

# LLMChain 인스턴스 생성
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 구성
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": llm_chain
}

C:\Users\jh838\AppData\Local\Temp\ipykernel_1920\3069750662.py:42: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=contextual_prompt)


## 10. 챗봇 구동

In [ ]:
# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    
    # Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})
    
    # 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })
    
    # LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content)

## RAG가 필요한 이유

RAG 체인을 활용하면, 질문과 관련된 문서를 검색하고 이를 기반으로 응답을 생성하기 때문에 보다 맥락에 맞고 최신 정보를 포함한 답변을 생성수 있다..
